In [1]:
import torch
device = torch.device("mps")
device

device(type='mps')

In [ ]:
pip install -U albumentations


In [2]:
import os
from PIL import Image

In [3]:
def make_ppm(source_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        if filename.endswith(".png"):
            file_path = os.path.join(source_dir, filename)
            with Image.open(file_path) as img:    
                output_filename = os.path.splitext(filename)[0] + '.ppm'
                output_path = os.path.join(output_dir, output_filename)
                img.save(output_path, 'PPM')
    print("Conversion complete.")
    
input = "../gtsrb-german-traffic-sign/Test"
output = "./input/GTSRB_Final_Test_Images/GTSRB/Final_Test/baseline_pgd_attack/all_ppm"
make_ppm(input, output)


Conversion complete.


In [4]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from glob import glob
from torchvision.transforms.functional import to_pil_image, to_tensor
from torch.nn import functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
from src.model import build_model  # Ensure this is correctly defined elsewhere

# Define the device to use for Tensor computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def gaussian_blur(image, kernel_size=5, sigma=2):
    """
    Applies Gaussian blurring using OpenCV.
    """
    image_np = image.squeeze(0).permute(1, 2, 0).numpy()
    image_np = (image_np * 255).astype(np.uint8)  # Scale to 0-255 range for CV2
    blurred_np = cv2.GaussianBlur(image_np, (kernel_size, kernel_size), sigmaX=sigma)
    blurred_tensor = to_tensor(blurred_np).unsqueeze(0)
    return blurred_tensor

def pgd_attack(model, image, epsilon, alpha, num_steps, y_true, targeted=False, random_start=True):
    model.eval()
    perturbed_image = image.clone().detach()
    if random_start:
        perturbed_image += (2 * epsilon * torch.rand_like(image) - epsilon)
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
    perturbed_image.requires_grad = True
    for _ in range(num_steps):
        outputs = model(perturbed_image)
        model.zero_grad()
        loss = F.cross_entropy(outputs, y_true)
        if targeted:
            loss = -loss
        loss.backward()
        with torch.no_grad():
            perturbed_image.data += alpha * perturbed_image.grad.sign()
            perturbation = torch.clamp(perturbed_image - image, -epsilon, epsilon)
            perturbed_image.data = torch.clamp(image + perturbation, 0, 1)
        perturbed_image.grad.zero_()
    return perturbed_image.detach()

def fgsm_attack(model, x, y, epsilon, iterations=1):
    model.eval()
    x_adv = x.clone().detach().requires_grad_(True)
    output = model(x_adv)
    loss = F.cross_entropy(output, y)
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        x_adv += epsilon * x_adv.grad.sign()
        x_adv = torch.clamp(x_adv, 0, 1)
    return x_adv.detach()

def createAdversarialImages():
    sign_names_df = pd.read_csv('./input/signnames.csv')
    gt_df = pd.read_csv('./input/GTSRB_Final_Test_GT/GT-final_test_full.csv', delimiter=',')
    gt_df = gt_df.set_index('Filename', drop=True)
    model = build_model(pretrained=False, fine_tune=False, num_classes=43).to(device)
    model.eval()
    model.load_state_dict(torch.load('./outputs/model.pth', map_location=device)['model_state_dict'])
    transform = A.Compose([ToTensorV2()])
    base_dir = './input/GTSRB_Final_Test_Images/GTSRB/Final_Test/baseline_pgd_attack'
    ppm_dir = os.path.join(base_dir, "all_ppm")
    blur_dir = os.path.join(base_dir, 'gaussian_blur')
    pgd_dir = os.path.join(base_dir, 'pgd_attack')
    fgsm_dir = os.path.join(base_dir, 'fgsm_attack')
    os.makedirs(blur_dir, exist_ok=True)
    os.makedirs(pgd_dir, exist_ok=True)
    os.makedirs(fgsm_dir, exist_ok=True)
    all_images = glob(os.path.join(ppm_dir,'*.ppm'))
    i = 0
    j = 0
    while j < 500:
        image_path = all_images[i]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32) / 255.0
        image_tensor = transform(image=image)['image'].unsqueeze(0).to(device)
        i += 1
        try:
            y_true = torch.tensor([gt_df.loc[os.path.basename(image_path), 'ClassId']]).to(device)
        except:
            print(f"{os.path.basename(image_path)} not in CSV")
        else:
            blurred_tensor = gaussian_blur(image_tensor)
            blurred_image = to_pil_image(blurred_tensor.squeeze(0))
            blurred_image.save(os.path.join(blur_dir, os.path.basename(image_path)), format='PPM') 
            pgd_tensor = pgd_attack(model, image_tensor, 0.1, 0.04, 10, y_true)
            pgd_image = to_pil_image(pgd_tensor.squeeze(0))
            pgd_image.save(os.path.join(pgd_dir, os.path.basename(image_path)), format='PPM')
            j += 1
        # fgsm_tensor = fgsm_attack(model, image_tensor, y_true, 0.1, 1)
        # fgsm_image = to_pil_image(fgsm_tensor.squeeze(0))
        # fgsm_image.save(os.path.join(fgsm_dir, os.path.basename(image_path)), format='PPM')

# To execute the function
createAdversarialImages()


[INFO]: Not loading pre-trained weights
[INFO]: Freezing hidden layers...
06578.ppm not in CSV
11004.ppm not in CSV
01217.ppm not in CSV
00109.ppm not in CSV
09893.ppm not in CSV


/Users/artijain/anaconda3/envs/deeplearning/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/artijain/anaconda3/envs/deeplearning/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


08555.ppm not in CSV
03400.ppm not in CSV
02078.ppm not in CSV
04409.ppm not in CSV
08233.ppm not in CSV
11762.ppm not in CSV
10468.ppm not in CSV
08227.ppm not in CSV
09139.ppm not in CSV
05703.ppm not in CSV
03372.ppm not in CSV
03414.ppm not in CSV
12519.ppm not in CSV
05065.ppm not in CSV
11010.ppm not in CSV
01203.ppm not in CSV
09887.ppm not in CSV
07672.ppm not in CSV
08541.ppm not in CSV
04353.ppm not in CSV
12531.ppm not in CSV
11038.ppm not in CSV
06544.ppm not in CSV
07882.ppm not in CSV
00135.ppm not in CSV
08569.ppm not in CSV
10326.ppm not in CSV
11986.ppm not in CSV
00653.ppm not in CSV
06222.ppm not in CSV
12257.ppm not in CSV
02044.ppm not in CSV
04435.ppm not in CSV
12243.ppm not in CSV
04421.ppm not in CSV
10454.ppm not in CSV
00647.ppm not in CSV
11992.ppm not in CSV
01559.ppm not in CSV
09105.ppm not in CSV
06236.ppm not in CSV
06550.ppm not in CSV
07896.ppm not in CSV
00121.ppm not in CSV
10332.ppm not in CSV
03428.ppm not in CSV
05059.ppm not in CSV
12525.ppm not

KeyboardInterrupt: 

In [6]:
import os
import csv
from PIL import Image

def filter_csv_by_images(folder_path, input_csv, output_csv):
    # Get a list of all image filenames in the folder
    image_files = os.listdir(folder_path)
    image_files = [f for f in image_files if f.lower().endswith(('.ppm'))]

    # Open input CSV file for reading
    with open(input_csv, 'r') as input_file:
        reader = csv.reader(input_file)
        # Open output CSV file for writing
        with open(output_csv, 'w', newline='') as output_file:
            writer = csv.writer(output_file)
            # Write the header row
            writer.writerow(next(reader))
            # Iterate through each row in the input CSV
            for row in reader:
                # Check if the image filename is in the list of image files
                if row[0] in image_files:
                    # Write the row to the output CSV
                    writer.writerow(row)

folder_path = './input/GTSRB_Final_Test_Images/GTSRB/Final_Test/gaussian_blur'
input_csv = './input/GTSRB_Final_Test_GT/GT-final_test_full.csv'
output_csv = './input/GTSRB_Final_Test_GT/GT-final_test_next.csv'

filter_csv_by_images(folder_path, input_csv, output_csv)



In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def visualize_images_in_grid_with_titles(filenames, titles):
    """
    Loads and visualizes multiple PPM images in a 2x2 grid with titles.
    
    Args:
        filenames (list of str): The paths to the PPM image files.
        titles (list of str): Titles for each image corresponding to their folders.
    """
    fig, axs = plt.subplots(2, 2, figsize=(4, 4))  # Set up a 2x2 grid of subplots
    axs = axs.ravel()  # Flatten the 2x2 grid to a 1D array for easier indexing
    
    for i, (filename, title) in enumerate(zip(filenames, titles)):
        image = Image.open(filename)
        axs[i].imshow(image)
        axs[i].set_title(title)  # Set the title for each subplot
        axs[i].axis('off')  # Turn off axis numbers and ticks for each subplot

    plt.tight_layout()
    plt.show()

# Example usage
base_path = "/Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/input/GTSRB_Final_Test_Images/GTSRB/Final_Test"  # Change this to the correct base path
image_name = "11776.ppm"  # Your image name

folders = ["Images", "gaussian_blur", "fgsm_attack", "pgd_attack"]
titles = ["Original Image", "Gaussian Blur", "FGSM Attack", "PGD Attack"]  # Titles based on folders
file_paths = [f"{base_path}/{folder}/{image_name}" for folder in folders]

visualize_images_in_grid_with_titles(file_paths, titles)


In [ ]:
!python cam.py gaussian_blur

In [ ]:
!python cam.py pgd_attack


In [ ]:
!python cam.py fgsm_attack

In [ ]:
import os
import shutil

# Define the paths to the directories
base_dir = '../input/GTSRB_Final_Test_Images/GTSRB/Final_Test'
gaussian_blur_dir = os.path.join(base_dir, 'gaussian_blur')
images_dir = os.path.join(base_dir, 'Images')
test_orig_dir = os.path.join(base_dir, 'original_subset')

# Ensure the target directory exists
os.makedirs(test_orig_dir, exist_ok=True)

# Get a list of filenames from the gaussian_blur directory
gaussian_files = os.listdir(gaussian_blur_dir)
gaussian_filenames = [os.path.splitext(file)[0] for file in gaussian_files]

# Copy files from the Images directory to test_orig if they exist in gaussian_blur
for filename in gaussian_filenames:
    src_path = os.path.join(images_dir, filename + '.ppm')
    dst_path = os.path.join(test_orig_dir, filename + '.ppm')
    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
        print(f"Copied {src_path} to {dst_path}")
    else:
        print(f"File does not exist: {src_path}")


In [ ]:
pwd

In [ ]:
!python cam.py original


In [ ]:
# %cd '/Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/input/GTSRB_Final_Test_Images/GTSRB/Final_Test/fgsm_attack'
# !rm *.ppm
# %cd '/Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/input/GTSRB_Final_Test_Images/GTSRB/Final_Test/gaussian_blur'
# !rm *.ppm
# %cd '/Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/input/GTSRB_Final_Test_Images/GTSRB/Final_Test/pgd_attack'
# !rm *.ppm
# %cd '/Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/input/GTSRB_Final_Test_Images/GTSRB/Final_Test/original_subset'
# !rm *.ppm
# %cd /Users/gman/Desktop/Traffic_Sign_Recognition_using_PyTorch_and_Deep_Learning/src

# Convert .ppm to .jpg (Optional)

In [ ]:
import cv2
import os
import glob

def convert_ppm_to_jpg(source_dir, target_dir):
    """
    Converts all PPM images in the specified directory to JPG format and saves them in a new directory.
    
    Args:
        source_dir (str): The directory containing PPM images.
        target_dir (str): The directory to save converted JPG images.
    """
    # Check if the source directory exists
    if not os.path.exists(source_dir):
        print(f"Source directory '{source_dir}' does not exist.")
        return

    # Create target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created directory '{target_dir}'.")

    # Glob to find all ppm files in the source directory
    ppm_files = glob.glob(os.path.join(source_dir, '*.ppm'))
    if not ppm_files:
        print(f"No .ppm files found in '{source_dir}'.")
        return

    for ppm_path in ppm_files:
        try:
            # Read the image using cv2
            image = cv2.imread(ppm_path)
            if image is None:
                print(f"Failed to read image from '{ppm_path}'.")
                continue

            # No need to convert to RGB as we are saving directly to JPEG which expects BGR
            # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Generate new file path with the .jpg extension
            base_name = os.path.basename(ppm_path)
            jpg_name = os.path.splitext(base_name)[0] + '.jpg'
            jpg_path = os.path.join(target_dir, jpg_name)
            
            # Save the image in JPG format
            cv2.imwrite(jpg_path, image)  # Save directly using the original BGR format
            print(f"Converted and saved {jpg_name}")
        except Exception as e:
            print(f"An error occurred processing {ppm_path}: {e}")


source_directory = '../input/GTSRB_Final_Test_Images/GTSRB/Final_Test/Images'  # adjust path as needed
target_directory = '../input/GTSRB_Final_Test_Images/GTSRB/Final_Test/Images_jpg'

convert_ppm_to_jpg(source_directory, target_directory)


# Visualize .ppm image ( Just one )

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt

# def visualize_ppm(filename):
#     """
#     Loads and visualizes a PPM image.
    
#     Args:
#         filename (str): The path to the PPM image file.
#     """
#     image = Image.open(filename)
#     plt.imshow(image)
#     plt.axis('off')  # Turn off axis numbers and ticks
#     plt.show()
# print("Original")
# visualize_ppm('../input/GTSRB_Final_Test_Images/GTSRB/Final_Test/Images/00000.ppm')




In [ ]:
# ##################          Gaussian Blur             ################## 

# import cv2
# import numpy as np
# import torch
# from torchvision.transforms.functional import to_tensor, to_pil_image

# def gaussian_blur(image, kernel_size=5, sigma=0):
#     """
#     Applies Gaussian blurring using OpenCV.
    
#     Args:
#         image (torch.Tensor): The input image tensor.
#         kernel_size (int): The size of the Gaussian kernel. Default is 5.
#         sigma (float): The standard deviation of the Gaussian kernel. Default is 0.
        
#     Returns:
#         torch.Tensor: The blurred image tensor.
#     """
#     # Convert tensor to numpy array
#     image_np = image.squeeze(0).permute(1, 2, 0).numpy()
#     image_np = (image_np * 255).astype(np.uint8)  # Scale to 0-255 range for CV2

#     # Apply Gaussian Blur using OpenCV
#     blurred_np = cv2.GaussianBlur(image_np, (kernel_size, kernel_size), sigmaX=sigma)

#     # Convert back to tensor
#     blurred_tensor = to_tensor(blurred_np).unsqueeze(0)
#     return blurred_tensor



# ##################          PGD attack             ################## 

# import torch
# import torch.nn.functional as F

# def pgd_attack(model, image, epsilon, alpha, num_steps, y_true, targeted=False):
#     """
#     Perform a PGD attack on an input image.

#     Args:
#         model (torch.nn.Module): The neural network model.
#         image (torch.Tensor): Input image tensor (should be normalized between 0 and 1).
#         epsilon (float): Maximum perturbation amount.
#         alpha (float): Step size per iteration.
#         num_steps (int): Number of iterations to perform the attack.
#         y_true (torch.Tensor): True labels for the image.
#         targeted (bool): Set to True for targeted attacks, False for untargeted.

#     Returns:
#         torch.Tensor: An adversarial example created from the input image.
#     """
#     model.eval()  # Ensure the model is in eval mode for consistent behavior
#     perturbed_image = image.clone().detach().requires_grad_(True)  # Initialize the perturbed image

#     for _ in range(num_steps):
#         outputs = model(perturbed_image)
#         model.zero_grad()  # Zero gradients to avoid accumulation

#         # Calculate the loss
#         loss = F.cross_entropy(outputs, y_true)
#         if targeted:
#             loss = -loss  # Maximize the loss for targeted attack
#         # Compute gradients
#         loss.backward()

#         # Apply PGD formula to adjust the image
#         with torch.no_grad():
#             # Calculate the step using the sign of the gradients
#             perturbed_image.data += alpha * perturbed_image.grad.sign()
#             # Project back into the epsilon-ball and clip to valid image range
#             perturbed_image.data = torch.clamp(perturbed_image.data, image - epsilon, image + epsilon)
#             perturbed_image.data = torch.clamp(perturbed_image.data, 0, 1)

#         # Detach and re-attach requires_grad to update gradients in next iteration
#         perturbed_image = perturbed_image.detach().requires_grad_(True)

#     return perturbed_image.detach()  # Return the perturbed image detached from the computation graph



# ##################          fgsm_attack             ################## 
# def fgsm_attack(model, x, y, epsilon, iterations):
#     """
#     Performs an untargeted adversarial attack using the FGSM method.

#     Args:
#         model (torch.nn.Module): The neural network model.
#         x (torch.Tensor): Input image tensor (should be normalized between 0 and 1).
#         y (torch.Tensor): True labels corresponding to x.
#         epsilon (float): The perturbation magnitude.
#         iterations (int): Number of iterations to apply the perturbation (usually 1 for FGSM).

#     Returns:
#         torch.Tensor: Adversarial image tensor with the same shape as input x.
    
#     """
#     model.eval()  # Set model to evaluation mode
#     x_adv = x.clone().detach().requires_grad_(True)  # Create a copy of x to modify
#     y = y.clone().detach()

#     for _ in range(iterations):
#         output = model(x_adv)
#         loss = F.cross_entropy(output, y)
#         model.zero_grad()  # Zero out previous gradients
#         loss.backward()  # Compute gradients with respect to input image

#         # Apply FGSM attack by adjusting the image pixels in the direction of the gradients
#         with torch.no_grad():
#             x_adv = x_adv + epsilon * x_adv.grad.sign()
#             x_adv = torch.clamp(x_adv, 0, 1)  # Ensure the adversarial image is still valid

#         # Reset requires_grad to True to compute new gradients in next iteration if necessary
#         x_adv = x_adv.detach().requires_grad_(True)

#     return x_adv

# # Example usage:
# # Assuming 'model' is loaded and set up, 'images' and 'labels' are prepared tensors
# # model = load_pretrained_model()
# # images, labels = load_data()
# # epsilon = 0.01  # Perturbation magnitude
# # num_iter = 1  # FGSM typically uses 1 iteration
# # adversarial_images = untargeted_attack(model, images, labels

In [ ]:
%cd outputs/test_results

In [ ]:
import os

# List of folders to create
list_operations = [
    "baseline_original_images",
    "baseline_gaussian_blur",
    "baseline_fgsm_attack",
    "baseline_pgd_attack",
    "de-gaussian_blur",
    "de-fgsm_attack",
    "de-pgd_attack",
    "de-fgsm_attack_light",
    "de-pgd_attack_light",
    "de-gaussian_blur_light"
]

# Specify the directory where you want to create the folders
base_directory = './'

# Create folders
for folder_name in list_operations:
    folder_path = os.path.join(base_directory, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    print(f"Created folder: {folder_path}")


In [ ]:
# baseline_original_images
# baseline_gaussian_blur
# baseline_fgsm_attack
# baseline_pgd_attack
# de-gaussian_blur
# de-fgsm_attack
# de-pgd_attack
# de-fgsm_attack_light
# de-pgd_attack_light
# de-gaussian_blur_light



In [ ]:
%cd /Users/gman/Desktop/Traffic_sign_clone_small/src

In [ ]:
!python cam.py de-gaussian_blur_light

In [ ]:
    gt_df = gt_df.set_index('Filename', drop=True)
